In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Carica i dati da CSV
data = pd.read_csv("bitcoin_data.csv")

# Prepara i dati per la rete neurale
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data["Value"].values.reshape(-1, 1))

# Dividi i dati in set di addestramento e test
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Crea le sequenze di input e output per la RNN
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 30  # Lunghezza della sequenza di input
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Crea il modello di rete neurale
model = Sequential()
model.add(LSTM(64, input_shape=(seq_length, 1)))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")

# Addestra il modello
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Valuta il modello
train_loss = model.evaluate(X_train, y_train, verbose=0)
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss di addestramento: {train_loss:.4f}")
print(f"Loss di test: {test_loss:.4f}")

# Effettua le previsioni
predicted_train = model.predict(X_train)
predicted_test = model.predict(X_test)

# Ripristina i valori originali
predicted_train = scaler.inverse_transform(predicted_train)
y_train = scaler.inverse_transform(y_train)
predicted_test = scaler.inverse_transform(predicted_test)
y_test = scaler.inverse_transform(y_test)

# Visualizza i risultati
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(data["Value"], label="Dati originali")
plt.plot(range(seq_length, len(predicted_train) + seq_length), predicted_train, label="Previsto (addestramento)")
plt.plot(range(seq_length + len(predicted_train), len(data)), predicted_test, label="Previsto (test)")
plt.xlabel("Giorni")
plt.ylabel("Valore Bitcoin (euro)")
plt.legend()
plt.show()
